In [45]:
import time
from datetime import datetime, timedelta
import vk_api
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import json

# Функция для получения списка идентификаторов групп из файла
def get_group_ids(file_path):
    with open(file_path, 'r') as file:
        group_ids = [line.strip() for line in file]
    return group_ids

# Функция для получения или создания листа в таблице
def get_or_create_worksheet(spreadsheet, worksheet_name):
    try:
        worksheet = spreadsheet.worksheet(worksheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows=100, cols=30)
    return worksheet

# Функция для сбора статистики постов группы
def collect_stats(group_id, target_spreadsheet_key, num_days, min_post_count, max_post_count, token):
    vk_session = vk_api.VkApi(token=token)
    api = vk_session.get_api()

    # Авторизация в Google Sheets
    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('../../../access/credentials.json', scope)
    client = gspread.authorize(credentials)
    target_spreadsheet = client.open_by_key(target_spreadsheet_key)

    worksheet_name = group_id
    worksheet = get_or_create_worksheet(target_spreadsheet, worksheet_name)

    # Определение даты начала сбора статистики
    start_date = datetime.now().date() - timedelta(days=num_days)
    rows_to_append = []

    offset = 0
    post_count = 0
    error_occurred = False

    # Цикл сбора статистики постов
    while post_count < max_post_count:
        # Получение списка постов группы
        posts = api.wall.get(owner_id=group_id, count=100, offset=offset, extended=1)['items']
        if not posts:
            break

        # Сортировка постов по дате
        sorted_posts = sorted(posts, key=lambda x: x['date'], reverse=True)

        for post in sorted_posts:
            post_date = datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d %H:%M:%S')

            # Проверка, что дата публикации поста попадает в интервал сбора статистики
            if datetime.strptime(post_date, '%Y-%m-%d %H:%M:%S').date() >= start_date:
                post_link = f"https://vk.com/wall-{group_id[1:]}_{post['id']}"
                text = post['text']
                comments = post.get('comments', {}).get('count', 0)
                likes = post.get('likes', {}).get('count', 0)
                reposts = post.get('reposts', {}).get('count', 0)
                views = post.get('views', {}).get('count', 0)

                try:
                    # Получение дополнительной статистики поста
                    stats_list = api.stats.getPostReach(owner_id=group_id, post_ids=post['id'])
                    for stats in stats_list:
                        reach_stats = stats.get('reach', {})
                        hide = reach_stats.get('hide', 0)
                        join_group = reach_stats.get('join_group', 0)
                        links = reach_stats.get('links', 0)
                        reach_subscribers = reach_stats.get('reach_subscribers', 0)
                        reach_total = reach_stats.get('reach_total', 0)
                        reach_viral = reach_stats.get('reach_viral', 0)
                        reach_ads = reach_stats.get('reach_ads', 0)
                        report = reach_stats.get('report', 0)
                        to_group = reach_stats.get('to_group', 0)
                        unsubscribe = reach_stats.get('unsubscribe', 0)

                        data = [
                            datetime.now().strftime('%Y-%m-%d %H:%M:%S'), post_date, post_link, text, comments,
                            likes, reposts, views, hide, join_group, links, reach_subscribers, reach_total,
                            reach_viral, reach_ads, report, to_group, unsubscribe
                        ]

                        rows_to_append.append(data)
                        post_count += 1

                        if post_count >= max_post_count:
                            break

                except vk_api.exceptions.VkApiError as e:
                    if not error_occurred:
                        print(f'Для {group_id} не получить доп.стату: {e}')
                        error_occurred = True

                    data = [
                        datetime.now().strftime('%Y-%m-%d %H:%M:%S'), post_date, post_link, text, comments,
                        likes, reposts, views
                    ]

                    rows_to_append.append(data)
                    post_count += 1

                if post_count >= max_post_count:
                    break

        offset += 100

    # Запись собранной статистики в таблицу
    if rows_to_append:
        worksheet.clear()
        worksheet.update('A1:R1', [['Дата и время проверки', 'Дата публикации', 'Ссылка на пост', 'Текст',
                                    'Комментарии', 'Лайки', 'Репосты', 'Просмотры', 'Скрытия', 'Вступления в группу',
                                    'Ссылки', 'Охват подписчиков', 'Общий охват', 'Охват вирусный', 'Охват рекламный',
                                    'Жалобы', 'Переходы в группу', 'Отписки']])
        worksheet.append_rows(rows_to_append[::-1], value_input_option='USER_ENTERED')
        print(f'[{datetime.now()}] Записано {len(rows_to_append)} строк для {group_id}.')
    else:
        print(f'[{datetime.now()}] Нет данных для записи в таблицу.')

if __name__ == "__main__":
    # Загрузка конфигурационных данных
    with open('../../../access/config.json') as config_file:
        config = json.load(config_file)

    token = config['vk_token_My']
    target_spreadsheet_key = config['target_spreadsheet_key']
    num_days = config['num_days']
    min_post_count = config['min_post_count']
    max_post_count = config['max_post_count']

    # Авторизация в Google Sheets
    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('../../../access/credentials.json', scope)
    client = gspread.authorize(credentials)

    # Получение списка идентификаторов групп
    group_ids = get_group_ids('../../../access/groups.txt')

    # Сбор статистики для каждой группы
    for group_id in group_ids:
        collect_stats(group_id, target_spreadsheet_key, num_days, min_post_count, max_post_count, token)


Для -193643180 не получить доп.стату: [15] Access denied
[2023-07-02 22:02:22.930293] Записано 3 строк для -193643180.
Для -221114738 не получить доп.стату: [15] Access denied
[2023-07-02 22:02:30.010652] Записано 5 строк для -221114738.
